In [80]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [85]:
from langchain_ollama import ChatOllama
llama_chat = ChatOllama(base_url="https://ollama.com",
                        model="gpt-oss:120b-cloud",
                        temperature=0,
                        max_tokens=100)

#### Define PLaywright Web browser Tool

In [64]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
# Enable Async Operation in Notebook
import nest_asyncio
nest_asyncio.apply()

async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools_by_name = {tool.name: tool for tool in tools }
tools_by_name

{'click_element': ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/jyotisahoo/Library/Caches/ms-playwright/chromium-1194/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=141.0.7390.37>),
 'navigate_browser': NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/jyotisahoo/Library/Caches/ms-playwright/chromium-1194/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=141.0.7390.37>),
 'previous_webpage': NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/jyotisahoo/Library/Caches/ms-playwright/chromium-1194/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=141.0.7390.37>),
 'extract_text': ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/jyotisahoo/Library/Caches/ms-playwright/chromium-1194/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=141.0.7390.37>),
 'extract_hyperlinks': ExtractHyperlinksToo

#### Test Tool Invoke

In [ ]:
navigation_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]
await navigation_tool.ainvoke({"url": "http://eaapp.somee.com/Employee"})
await get_elements_tool.ainvoke({"selector": "td", "attributes": ["innerText"]})

#### Agent to invoke Playwright Browser Tool

In [87]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

browser_agent = create_agent(model=llama_chat,
                             tools=tools,
                             system_prompt="""
                             You are a helpful assistant that can answer question using the provided tools.
                             If the question involves web scraping, use appropriate tool to get the data.
                             Always structure the final response in JSON format.
                             """, 
                             debug=True)
query = ("Navigate to https://www.apple.com/ page."
         "Wait for the page to load completely."
         "Once page is loaded get me the latest iPhone model name, Watch model and iPad model from the page.")
human_message = HumanMessage(content=query)
result = await browser_agent.ainvoke({"messages": [human_message]})
print(result["messages"][-1].content)

[values] {'messages': [HumanMessage(content='Navigate to https://www.apple.com/ page.Wait for the page to load completely.Once page is loaded get me the latest iPhone model name, Watch model and iPad model from the page.', additional_kwargs={}, response_metadata={}, id='f0fb74d4-8f2a-4025-aa37-de42a7caad30')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b', 'created_at': '2025-12-17T17:26:31.621610552Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1255361804, 'load_duration': None, 'prompt_eval_count': 283, 'prompt_eval_duration': None, 'eval_count': 180, 'eval_duration': None, 'logprobs': None, 'model_name': 'gpt-oss:120b', 'model_provider': 'ollama'}, id='lc_run--019b2d59-5dbb-7563-82a5-ba87daa748ff-0', tool_calls=[{'name': 'navigate_browser', 'args': {'url': 'https://www.apple.com/'}, 'id': '60aeb196-ed73-4b6d-9262-f4f7929dcdef', 'type': 'tool_call'}], usage_metadata={'input_tokens': 283, 'output_to

In [86]:
from langchain.agents import create_agent
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit

# 1. Initialize Tools (Keep it slim for Qwen 8B)
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
all_tools = toolkit.get_tools()
# Filter for just the essentials so the small model doesn't get overwhelmed
tools = [t for t in all_tools if t.name in ["navigate_browser", "get_elements"]]

# 2. Create the Agent using the new v1.0 standard
# The 'system_prompt' parameter replaces the old 'state_modifier'
browser_agent = create_agent(
    model=llama_chat, 
    tools=tools,
    system_prompt=(
        "You are a web scraper. To find product names on Apple.com:\n"
        "1. Navigate to the page.\n"
        "2. Use 'get_elements' with selector 'h1, h2' to find the main product titles.\n"
        "3. Return a JSON object like: {'iphone': '...', 'watch': '...', 'ipad': '...'}.\n"
        "Do not return any conversational text, ONLY the JSON."
    )
)

# 3. Execution
# In v1.x, you invoke with a simple dictionary. 
# It handles the loop (Thought/Action/Observation) automatically.
query = "Get the latest iPhone, Watch, and iPad models from apple.com."
result = await browser_agent.ainvoke({"messages": [("user", query)]})

# 4. Result
print(result["messages"][-1].content)

{
  "iphone": "iPhone 15 Pro",
  "watch": "Apple Watch Series 9",
  "ipad": "iPad Pro (6th generation)"
}
